# Analysis of GS-15 employees at DOJ

The analysis in this notebook counts the number of Department of Justice employees who, as of the end of March 2017, had attained a pay grade of `GS-15` the highest grade on the General Schedule pay scale.

The data comes from the "__FedScope Employment Cube (March 2017)__" file [published by the Office of Personnel Management](https://www.opm.gov/data/Index.aspx?tag=FedScope).

## Load the data

In [1]:
import pandas as pd

#### Employees

In [2]:
employees = pd.read_csv(
    "../data/fedscope-data-2017-03/FACTDATA_MAR2017.TXT",
    low_memory=False
)
employees.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,K,21,15,J,0905,1,GS-15,P,XXXX,8,30,F,1,201703,1,161900.0,42.4
1,AA00,11,G,15,NaN,D,0301,2,ES-**,Q,XXXX,2,50,F,1,201703,1,172100.0,7.4
2,AA00,11,H,15,NaN,G,0905,1,ES-**,Q,XXXX,2,50,F,1,201703,1,172100.0,24.2
3,AA00,11,C,15,12,A,0905,1,GS-12,G,XXXX,8,30,F,1,201703,1,79720.0,0.9
4,AA00,11,C,15,12,A,0905,1,GS-12,G,XXXX,8,30,F,1,201703,1,79720.0,0.8


#### Occupation codes

In [3]:
occupation_codes = pd.read_csv("../data/fedscope-data-2017-03/DTocc.txt")\
    .set_index("OCC")[["OCCT"]]
occupation_codes.head()

,OCCT
OCC,
0006,0006-CORRECTIONAL INSTITUTION ADMINISTRATION
0007,0007-CORRECTIONAL OFFICER
0017,0017-EXPLOSIVES SAFETY
0018,0018-SAFETY AND OCCUPATIONAL HEALTH MANAGEMENT
0019,0019-SAFETY TECHNICIAN


#### Subagency codes

In [4]:
subagency_codes = pd.read_csv("../data/fedscope-data-2017-03/DTagy.txt")\
    .set_index("AGYSUB")[["AGYSUBT"]]
subagency_codes.head()

,AGYSUBT
AGYSUB,
AF02,AF02-AIR FORCE INSPECTION AGENCY (FO)
AF03,AF03-AIR FORCE OPERATIONAL TEST AND EVALUATION...
AF06,AF06-AIR FORCE AUDIT AGENCY
AF07,AF07-AIR FORCE OFFICE OF SPECIAL INVESTIGATIONS
AF09,AF09-AIR FORCE PERSONNEL CENTER


#### Work status codes

In [5]:
work_status_codes = pd.read_csv("../data/fedscope-data-2017-03/DTwkstat.txt")\
    .set_index("WORKSTAT")[["WORKSTATT"]]
work_status_codes.head()

,WORKSTATT
WORKSTAT,
1,Non-Seasonal Full Time Permanent
2,Other Employees


## Select just the DOJ GS-15s

(The agency code for the Department of Justice is `DJ`.)

In [6]:
doj_gs_15 = employees[
    (employees["AGYSUB"].str.slice(0, 2) == "DJ") &
    (employees["GSEGRD"] == "15")
][[ "OCC", "AGYSUB", "GSEGRD", "WORKSTAT" ]]\
    .set_index("OCC").join(occupation_codes)\
    .set_index("AGYSUB").join(subagency_codes)\
    .set_index("WORKSTAT").join(work_status_codes)
    
doj_gs_15.head()

,GSEGRD,OCCT,AGYSUBT,WORKSTATT
WORKSTAT,,,,
1,15,0080-SECURITY ADMINISTRATION,"DJ01-OFFICES, BOARDS AND DIVISIONS",Non-Seasonal Full Time Permanent
1,15,0080-SECURITY ADMINISTRATION,"DJ01-OFFICES, BOARDS AND DIVISIONS",Non-Seasonal Full Time Permanent
1,15,0080-SECURITY ADMINISTRATION,"DJ01-OFFICES, BOARDS AND DIVISIONS",Non-Seasonal Full Time Permanent
1,15,0080-SECURITY ADMINISTRATION,"DJ01-OFFICES, BOARDS AND DIVISIONS",Non-Seasonal Full Time Permanent
1,15,0080-SECURITY ADMINISTRATION,"DJ01-OFFICES, BOARDS AND DIVISIONS",Non-Seasonal Full Time Permanent


There were 6,894 employees who met these criteria:

In [7]:
len(doj_gs_15)

6894

... 6,315 of which were [non-seasonal full-time employees with "no absolute end date"](https://www.opm.gov/FAQs/QA.aspx?fid=56538f91-625a-4333-84ba-28b3574b7942&pid=1e2b48ff-5289-4b5c-a318-b2a22134e48c):

In [8]:
doj_gs_15["WORKSTATT"].value_counts()

Non-Seasonal Full Time Permanent    6315
Other Employees                      579
Name: WORKSTATT, dtype: int64

## Count employees by subagency and occupation

In [9]:
counts = doj_gs_15.groupby([ "AGYSUBT", "OCCT" ]).size()\
    .sort_values(ascending=False).to_frame("count").reset_index()\
    .sort_values([ "count", "AGYSUBT", "OCCT" ], ascending=[ False, True, True ])\
    [["AGYSUBT", "OCCT", "count"]]

counts.to_csv("../output/doj-gs-15-by-subagency-and-occupation.csv", index=False)
counts.head()

,AGYSUBT,OCCT,count
0,"DJ01-OFFICES, BOARDS AND DIVISIONS",0905-GENERAL ATTORNEY,2958
1,DJ02-FEDERAL BUREAU OF INVESTIGATION,1811-CRIMINAL INVESTIGATION,627
2,DJ11-U.S. TRUSTEE PROGRAM,0905-GENERAL ATTORNEY,243
3,DJ06-DRUG ENFORCEMENT ADMINISTRATION,1811-CRIMINAL INVESTIGATION,237
4,DJ03-BUREAU OF PRISONS/FEDERAL PRISON SYSTEM,0602-MEDICAL OFFICER,210


In [10]:
subagency_counts = counts.groupby("AGYSUBT")["count"].sum()\
    .reset_index()\
    .sort_values([ "count", "AGYSUBT" ], ascending=[ False, True ])
    
subagency_counts.to_csv("../output/doj-gs-15-by-subagency.csv", index=False)
subagency_counts.head()

,AGYSUBT,count
0,"DJ01-OFFICES, BOARDS AND DIVISIONS",3302
1,DJ02-FEDERAL BUREAU OF INVESTIGATION,1493
3,DJ06-DRUG ENFORCEMENT ADMINISTRATION,456
2,DJ03-BUREAU OF PRISONS/FEDERAL PRISON SYSTEM,441
8,DJ11-U.S. TRUSTEE PROGRAM,275


In [11]:
occupation_counts = counts.groupby("OCCT")["count"].sum()\
    .reset_index()\
    .sort_values([ "count", "OCCT" ], ascending=[ False, True ])
    
occupation_counts.to_csv("../output/doj-gs-15-by-occupation.csv", index=False)
occupation_counts.head()

,OCCT,count
37,0905-GENERAL ATTORNEY,3916
65,1811-CRIMINAL INVESTIGATION,1059
68,2210-INFORMATION TECHNOLOGY MANAGEMENT,263
18,0343-MANAGEMENT AND PROGRAM ANALYSIS,238
27,0602-MEDICAL OFFICER,214


---

---

---